In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
from pathlib import Path
import json
from random import randint
from collections import Counter
from collections import defaultdict

In [4]:
import json 
from PIL import Image, ImageDraw
import PIL
from IPython.display import display
from shapely import wkt
from shapely.geometry.multipolygon import MultiPolygon

In [5]:
import cv2
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from tifffile import imread

In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
from copy import copy 

%matplotlib inline
# figure size in inches optional
rcParams['figure.figsize'] = 30, 30
plt.rcParams['legend.title_fontsize'] = 14

In [7]:
# Fetch all post disaster json files
labels_generator = Path('data/test/labels').rglob(pattern=f'*pre_*.json')

In [8]:
# Group json files by disaster
def get_disaster_dict(labels_generator):    
    disaster_dict = defaultdict(list)
    for label in labels_generator:
        disaster_type = label.name.split('_')[0]
        disaster_dict[disaster_type].append(str(label))
    return disaster_dict

In [9]:
disaster_dict = get_disaster_dict(labels_generator)

In [10]:
# List of Disasters present in the dataset
disaster_dict.keys()

dict_keys(['guatemala-volcano', 'hurricane-florence', 'hurricane-harvey', 'hurricane-matthew', 'hurricane-michael', 'mexico-earthquake', 'midwest-flooding', 'palu-tsunami', 'santa-rosa-wildfire', 'socal-fire'])

In [11]:
# Select a particular disaster
disaster_labels = disaster_dict['hurricane-matthew']

In [12]:
def read_label(label_path):
    with open(label_path) as json_file:
        image_json = json.load(json_file)
        return image_json


In [13]:
# Color codes for polygons
damage_dict = {
    "no-damage": (255, 255, 255),
    "minor-damage": (255, 255, 255),
    "major-damage": (255, 255, 255),
    "destroyed": (255, 255, 255),
    "un-classified": (255, 255, 255)
}

In [14]:
# Damage to integer mapping
damage_dict_multi = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 0
}

In [15]:
def get_damage_type(properties):
    if 'subtype' in properties:
        return properties['subtype']
    else:
        return 'no-damage'

In [16]:
def annotate_img(draw, coords):
        wkt_polygons = []

        for coord in coords:
            damage = get_damage_type(coord['properties'])
            wkt_polygons.append((damage, coord['wkt']))

        polygons = []

        for damage, swkt in wkt_polygons:
            polygons.append((damage, wkt.loads(swkt)))

        for damage, polygon in polygons:
            x,y = polygon.exterior.coords.xy
            coords = list(zip(x,y))
            draw.polygon(coords, damage_dict[damage])

        del draw

In [17]:
def annotate_img_multi(draw, coords):
    wkt_polygons = []

    for coord in coords:
        damage = get_damage_type(coord['properties'])
        wkt_polygons.append((damage, coord['wkt']))

    polygons = []

    for damage, swkt in wkt_polygons:
        polygons.append((damage, wkt.loads(swkt)))

    for damage, polygon in polygons:
        x, y = polygon.exterior.coords.xy
        coords = list(zip(x, y))
        
        draw.polygon(coords, fill=damage_dict_multi[damage])


In [18]:
def get_mask(json_path):
    image_json = read_label(json_path)
    img_name = image_json['metadata']['img_name']
  
    img = Image.new('RGB', (1024, 1024), color='black')
    draw = ImageDraw.Draw(img, 'RGBA')
    annotate_img(draw, image_json['features']['xy'])

    return img


In [19]:
def get_mask_multi(json_path):
    image_json = read_label(json_path)
    img_name = image_json['metadata']['img_name']



    img = Image.new('L', (1024, 1024), color=0)  
    draw = ImageDraw.Draw(img)
    annotate_img_multi(draw, image_json['features']['xy'])

    return img

In [39]:
# # Loop through each JSON path in disaster_labels
labels_generator = Path('data\\test\\labels').rglob(pattern=f'*post*.json')
disaster_dict = get_disaster_dict(labels_generator)

c = 0
flooding_subset = []
for i in disaster_dict.keys():
    disaster_labels = disaster_dict[i]
    for j in disaster_labels:
        disaster_type = read_label(j)['metadata']['disaster_type']
        if disaster_type == "flooding":
            img_path = j.replace('labels', 'images').replace('json', 'tif')
            flooding_subset.append(img_path)
flooding_subset = np.array(flooding_subset)
print(flooding_subset.shape)



(296,)


In [40]:
print(flooding_subset[0])

data\test\images\hurricane-florence_00000004_post_disaster.tif


In [41]:
for path in flooding_subset:

    image = tifffile.imread(str(path)) 
    base = path[17:]
    path = 'data\\test_set\images\\' + base[:-4] + '.png'
    cv2.imwrite(path, image)
    


In [53]:
np.load('data\damage_classification_train\multiclass_segmentation_labels\\'+base_name+'.npy').max()

0

In [20]:
labels_generator = Path('data/tier3/labels').rglob(pattern=f'*post_*.json')
disaster_dict = get_disaster_dict(labels_generator)
c = 0
flooding_subset = []
for i in disaster_dict.keys():
    disaster_labels = disaster_dict[i]
    for j in disaster_labels:
        disaster_type = read_label(j)['metadata']['disaster_type']
        if disaster_type == "flooding":
            flooding_subset.append(j)
print(flooding_subset[0])

data\tier3\labels\nepal-flooding_00000000_post_disaster.json


In [67]:
for json_path in flooding_subset:
    # Get corresponding image path
    img_path = json_path.replace('labels', 'images').replace('json', 'tif')
    # Read the image
    image = tifffile.imread(img_path)
    # Extract filename (without extension) to use for saving
    base_name = os.path.basename(json_path).replace('.json', '')
    # Save the image and mask
    tifffile.imsave(os.path.join('data/damage_classification_train/images', base_name + '.tif'), image)
    


C:\Users\rajsh\AppData\Local\Temp\ipykernel_19856\3782621949.py:9: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join('data/damage_classification_train/images', base_name + '.tif'), image)


In [68]:
# Copy the JSON files to the destination directory
for file_path in flooding_subset:
    # Read the JSON content from the original file
    with open(file_path, 'r') as file:
        json_content = json.load(file)
    
    # Define the new path where the JSON file will be saved
    new_file_path = 'data\\damage_classification_train\\json_labels\\' + Path(file_path).name


    # Write the JSON content to the new file
    with open(new_file_path, 'w') as file:
        json.dump(json_content, file, indent=4)

In [62]:
import tifffile
from shapely.wkt import loads
from PIL import Image, ImageDraw
labels_generator = Path('data/damage_classification_train/json_labels').rglob(pattern=f'*post_*.json')
disaster_dict = get_disaster_dict(labels_generator)

def get_bounding_box(wkt_polygon, margin=5):
    """
    Calculate the bounding box for a given polygon and expand it by the specified margin.
    :param wkt_polygon: WKT representation of the polygon.
    :param margin: Amount by which to expand the bounding box on each side.
    :return: Coordinates of the expanded bounding box.
    """
    polygon = loads(wkt_polygon)

    minx, miny, maxx, maxy = polygon.bounds
    center_x = (minx + maxx) / 2
    center_y = (miny + maxy) / 2
    minx = center_x - 75
    maxx = center_x + 75
    miny = center_y - 75
    maxy = center_y + 75
    # Get the Center of the the box. Return b
    # Expand the bounding box
    return [(minx, miny), (maxx, maxy)]

def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)

def save_cropped_image(image, bbox, save_path):
    """Crop and save the image based on the bounding box."""
    cropped_image = image.crop((bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1]))
    cropped_image.save(save_path)

# Create directories for each damage type
base_dir = 'data/damage_classification_train/extracted_buildings'
nd = 0
mind = 0
majd = 0
destr = 0
unc = 0
# Process each image and crop the buildings
for i in disaster_dict.keys():
    disaster_labels = disaster_dict[i]
    for j in disaster_labels:
        disaster = read_label(j)
        img_path = j.replace('json_labels', 'images').replace('json', 'tif')
        image = tifffile.imread(img_path) 
        image_pil = Image.fromarray(np.array(image, dtype=np.uint8))  
        
        wkt_polygons = []
        for coord in disaster['features']['xy']:
            damage = get_damage_type(coord['properties'])
            wkt_polygons.append((damage, coord['wkt']))
        k = 0
        for damage, wkt_polygon in wkt_polygons:
            bbox = get_bounding_box(wkt_polygon)
            # Determine the save path based on damage type
            if damage == 'no-damage':
                save_path = os.path.join(base_dir, '0')
                nd = nd + 1
            elif damage == 'minor-damage':
                save_path = os.path.join(base_dir, '1')
                mind += 1
            elif damage == 'major-damage':
                save_path = os.path.join(base_dir, '2')
                majd += 1
            elif damage == 'destroyed':
                save_path = os.path.join(base_dir, '3')
                destr += 1
            else:
                save_path = os.path.join(base_dir, 'unclassified')
                unc += 1
        
            # Generate a unique filename for each cropped image
            filename = f"{i}_{os.path.basename(j).split('.')[0]}_{damage}"
            filename = filename + '_'+str(k) + '.png'
            k += 1
            save_cropped_image(image_pil, bbox, os.path.join(save_path, filename))

55465 8078 14323 1032 2583
0.6807108405640578 0.09913967673445342 0.17578331144684037 0.012665529387219106 0.03170064186742921


In [63]:
print(nd, mind, majd, destr, unc)
total = nd + mind + majd + destr + unc
print(nd/total, mind/total, majd/total, destr/total, unc/total)
print(nd/total+ mind/total+ majd/total+ destr/total+ unc/total)

55465 8078 14323 1032 2583
0.6807108405640578 0.09913967673445342 0.17578331144684037 0.012665529387219106 0.03170064186742921
1.0


In [160]:
import os
from pathlib import Path
import numpy as np
import tifffile
from shapely.wkt import loads
from PIL import Image

def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)

def save_cropped_image(image, bbox, save_path):
    """Crop and save the image based on the bounding box."""
    cropped_image = image.crop((bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1]))
    cropped_image.save(save_path)

# Create directories for each damage type
base_dir = 'data/damage_classification_train/extracted_buildings'
damage_dirs = ['0', '1', '2', 'unclassified']
for damage_dir in damage_dirs:
    create_directory(os.path.join(base_dir, damage_dir))

# Process each image and crop the buildings
for i in disaster_dict.keys():
    disaster_labels = disaster_dict[i]
    for j in disaster_labels:
        disaster = read_label(j)

        img_path = j.replace('json_labels', 'images').replace('json', 'tif')
        image = tifffile.imread(img_path) 
        image_pil = Image.fromarray(np.array(image, dtype=np.uint8))  
        
        wkt_polygons = []
        for coord in disaster['features']['xy']:
            damage = get_damage_type(coord['properties'])
            wkt_polygons.append((damage, coord['wkt']))

        for damage, wkt_polygon in wkt_polygons:
            bbox = get_bounding_box(wkt_polygon)
            # Determine the save path based on damage type
            if damage == 'no-damage':
                save_path = os.path.join(base_dir, '0')
            elif damage == 'minor-damage':
                save_path = os.path.join(base_dir, '1')
            elif damage == 'major-damage':
                save_path = os.path.join(base_dir, '2')
            else:
                save_path = os.path.join(base_dir, 'unclassified')
            
            # Generate a unique filename for each cropped image
            filename = f"{i}_{os.path.basename(j).split('.')[0]}_{damage}.png"
            save_cropped_image(image_pil, bbox, os.path.join(save_path, filename))

In [2]:
# Loop through each JSON path in disaster_labels
labels_generator = Path('data/tier1/labels').rglob(pattern=f'*pre_*.json')
disaster_dict = get_disaster_dict(labels_generator)
c = 0
flooding_subset = []
for i in disaster_dict.keys():
    disaster_labels = disaster_dict[i]
    for j in disaster_labels:
        disaster_type = read_label(j)['metadata']['disaster_type']
        if disaster_type == "flooding":
            flooding_subset.append(j)

labels_generator = Path('data/tier3/labels').rglob(pattern=f'*pre_*.json')
disaster_dict = get_disaster_dict(labels_generator)
for i in disaster_dict.keys():
    disaster_labels = disaster_dict[i]
    for j in disaster_labels:
        disaster_type = read_label(j)['metadata']['disaster_type']
        if disaster_type == "flooding":
            flooding_subset.append(j)



NameError: name 'Path' is not defined

In [67]:
json_path = flooding_subset[13]
print(json_path)
img_path = json_path.replace('labels', 'images').replace('json', 'tif').replace('post', 'pre')
img_path = 'data\damage_classification_train\images\hurricane-florence_00000389_post_disaster.tif'
print(img_path)
    # Read the image
image = tifffile.imread(img_path)/255.0
image = cv2.resize(image,(572,572))
cv2.imshow("sample", image)
cv2.waitKey()
cv2.destroyAllWindows()
# Get the mask using your function
mask = get_mask(json_path)
mask.show()
    

    

data\damage_classification_train\images\hurricane-florence_00000389_post_disaster.tif


In [ ]:
# Ensure folders exist
if not os.path.exists('data/train/images'):
    os.makedirs('data/train/images')
if not os.path.exists('data/train/masks'):
    os.makedirs('data/train/masks')


for json_path in flooding_subset:
    # Get corresponding image path
    img_path = json_path.replace('labels', 'images').replace('json', 'tif')
    # Read the image
    image = tifffile.imread(img_path)
    
    # Get the mask using your function
    mask = get_mask(json_path)
    
    # Extract filename (without extension) to use for saving
    base_name = os.path.basename(json_path).replace('.json', '')
    
    # Save the image and mask
    tifffile.imsave(os.path.join('data/train/images', base_name + '.tif'), image)
    
    # Assuming the mask is also in a format compatible with tifffile, save it
    # Modify as necessary depending on the mask format
    tifffile.imsave(os.path.join('data/train/masks', base_name + '_mask.tif'), mask)


In [42]:
print(len(os.listdir("data\\train\\masks")))
mask = tifffile.imread("data\\train\\masks\\hurricane-florence_00000000_pre_disaster_mask.tif")
print(mask.shape)
# Noise Removal


1536
(1024, 1024, 3)


In [89]:
def replace_third_underscore(s):
    parts = s.split('_')
    if len(parts) < 3:
        # If there are less than 4 parts, there's no third underscore to replace
        return s
    return '_'.join(parts[:3]) + '-' + '_'.join(parts[3:])


In [ ]:
import os

def replace_third_underscore(s):
    parts = s.split('_')
    if len(parts) < 4:
        return s
    return '_'.join(parts[:3]) + '-' + '_'.join(parts[3:])

target = 'final\\results'  # Directory with the original files

# Iterate over each file in the target directory
for path in os.listdir(target):
    # Construct the full path
    full_path = os.path.join(target, path)
    
    # Skip directories, only rename files
    if os.path.isfile(full_path):
        new_name = replace_third_underscore(path)
        new_full_path = os.path.join(target, new_name)
        
        # Rename the file
        os.rename(full_path, new_full_path)
        print(f"Renamed '{path}' to '{new_name}'")


In [93]:
import cv2
import os
import numpy as np
from pathlib import Path

# Define the directories
target_dir = Path('final/targets')
results_dir = Path('final/results')


# Function to normalize and save images
def normalize_and_replace_images(directory):
    for file in directory.iterdir():
        # Check if the file is an image and contains 'localization' in its name
        if file.suffix == '.png' and 'localization' in file.stem:
            # Read the image in grayscale
            img = cv2.imread(str(file), cv2.IMREAD_GRAYSCALE)

            # Normalize the image to the range [0, 1]
            normalized_img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            # Replace the original image with the normalized image
            cv2.imwrite(str(file), normalized_img)

# Normalize and replace images in both directories
normalize_and_replace_images(target_dir)
normalize_and_replace_images(results_dir)

"Normalization and replacement of images complete."






'Normalization and replacement of images complete.'